In [3]:
import pandas as pd
copy = pd.read_json("/dccstor/myu/experiments/eli5_cmsks_learnscore_hop2_ctx3_greedy_3e05_0802/eval_pred_scores.json", lines=True)
fid = pd.read_json("/dccstor/myu/experiments/eli5_fid_greedy_ctx3_0729/eval_pred_scores.json", lines=True)
kg_hop2 = pd.read_json('/dccstor/myu/data/kilt_eli5_dpr/eli5-dev-kilt-dpr-kg-hop2.json', lines=True)

In [6]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokens import Token
stop_words_getter = lambda token: token.is_stop or token.lower_ in STOP_WORDS \
                                                or token.lemma_ in STOP_WORDS
Token.set_extension('is_stop', getter=stop_words_getter, force=True)
nlp = spacy.load("en_core_web_lg", disable=["parser","ner"])
def get_nonstop_words(x):
    y = [
        token.lemma_ for token in nlp(x) if
        not token.is_stop
        and not token.is_currency
        and not token.is_digit
        and not token.is_punct
        and not token.is_space
        and not token.like_num
        and not token.pos_ == "PROPN"
    ]
    return list(set(y))

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def union(lst1, lst2):
    return list(set(lst1) & set(lst2))

def get_good_examples(name1, experiment1, name2, experiment2, hops):
    
    count = 0
    cnt_comparable_examples = 0
    for _, example in hops.iterrows():
        experiment_paragraph = experiment1[experiment1["id"] == example.id].iloc[0]
        experiment2_paragraph = experiment2[experiment2["id"] == example.id].iloc[0]
        # different best paragraphs
        if experiment2_paragraph['max_rouge_ref'] != experiment_paragraph['max_rouge_ref']:
            continue
        
        gold_words = get_nonstop_words(experiment_paragraph['max_rouge_ref'])
        generated_words = get_nonstop_words(experiment_paragraph['prediction_text'])
        generated_words2 = get_nonstop_words(experiment2_paragraph['prediction_text'])

        if len(example['kg_vocab']) == 0:
            continue
        cnt_comparable_examples += 1
        gold_intersection = intersection(gold_words,example['kg_vocab'])
        generated_intersection = intersection(generated_words,example['kg_vocab'])
        gen_gold = intersection(generated_words,gold_words)
        generated_intersection2 = intersection(generated_words2,example['kg_vocab'])
        gen_gold2 = intersection(generated_words2,gold_words)
        gen_gold_kg1 = intersection(gen_gold,example['kg_vocab'])
        gen_gold_kg2 = intersection(gen_gold2,example['kg_vocab'])

        # equal copy is higher than fid
        # #len(gold_intersection)/len(example['kg_vocab']) > .1 and len(generated_intersection)/len(example['kg_vocab']) and \        
        # if experiment_paragraph['rouge-L'] > experiment2_paragraph['rouge-L'] and \
        # len(generated_intersection) > len(generated_intersection2) and \
        # len(gen_gold_kg1) > 0:

        if experiment_paragraph['rouge-L'] < experiment2_paragraph['rouge-L'] and \
        len(generated_intersection) > len(generated_intersection2) and \
        len(gen_gold_kg1) > 0:
        # if \
        # len(generated_intersection) > len(generated_intersection2):
            count += 1
            print(example['id'] + " " + str(example['input']) + "\n--\ngold: " + experiment_paragraph['max_rouge_ref'] 
            + "\n--\nkg " + str(example['kg_vocab']) + 
            "\n--\ngenerated " + name1 + ": " + experiment_paragraph['prediction_text'] +
            "\n--\ngenerated " + name2 + ": " + experiment2_paragraph['prediction_text'])
            print("--\ngold/kg: " + str(gold_intersection) + " " + str(len(gold_intersection)/len(example['kg_vocab'])))
            print("--\nkg/gold: " + str(gold_intersection) + " " + str(len(gold_intersection)/len(gold_words)))
            print("gen/kg " + name1 + ": " + str(generated_intersection) + " " + str(len(generated_intersection)/len(example['kg_vocab'])))
            print("gen/kg " + name2 + ": " + str(generated_intersection2) + " " + str(len(generated_intersection2)/len(example['kg_vocab'])))
            print("gen/gold: " + str(gen_gold) + " " + str(len(gen_gold)/len(example['kg_vocab'])))
            print("gen/gold/kg " + name1 + ": " + str(gen_gold_kg1))
            print("gen/gold/kg " + name2 + ": " + str(gen_gold_kg2))
            print("rouge " + name1 + " " + str(experiment_paragraph['rouge-L']) + " > rouge " + name2 + " " + str(experiment2_paragraph['rouge-L']))
            print("------")

    print(count)
    print(cnt_comparable_examples)
    print(len(hops))

In [7]:
get_good_examples("scoring copy", copy, "fid", fid, kg_hop2)

3ngwkn Why don't large dogs consider small dogs prey, but they consider cats prey?
--
gold: I don't think they see cats as prey, either. I think they see cats as competitive predators. They aren't killing cats to eat them, they are killing cats to eliminate them.
--
kg ['particular', 'group', 'hand', 'bites', 'fast', 'properly', 'reared', 'preference', 'felines', 'phobias', 'people', 'involved', 'relationships', 'soft', 'portion', 'conditions', 'reacted', 'terms', 'crime', 'dogs', 'play', 'mammal', 'kill', 'cats', 'number', 'predators', 'self', 'social', 'cultural', 'members', 'presence', 'drive', 'severe', 'displayed', 'significant', 'language', 'complete', 'wild', 'aid', 'large', 'alerts', 'huntsmen', 'opportunity', 'non', 'energetic', 'mice', 'domesticated', 'bite', 'prairie', 'primarily', 'baywhich', 'term', 'catching', 'gamefox', 'predator', 'body', 'carnivorous', 'inflicted', 'guardians', 'short', 'wolves', 'identification', 'moose', 'university', 'home', 'hunters', 'human', '-',

In [10]:
get_good_examples("scoring copy", copy, "fid", fid, kg_hop2)

5fpy11 What caused racial division and discrimination between Hutus and Tutsis in Rwanda?
--
gold: So before the Europeans arrived Rwandan and Burundian society had a hierarchy with Tutsis at the top and Hutus and the twa at the bottom there was no clear racial distinction because most Rwandans have the same features and the system was primarily economic with the those who were rich considered Tutsis and those without considered Hutus. Then the Belgians came and colonized the region and they saw the hierarchical system as a way to effectively control the people. In doing so they added a racial dynamic in which those with lighter skin,narrow noses and tall height were considered Tutsi while those who had more negroid features were considered Hutu or twa. The Tutsis who ran Rwanda weren't very nice to the Hutus and this sowed the hatred and resentment for the Tutsis amongst Hutus. This changed with the wave of African independence movements which made the Tutsis demand independence(becau